In [ ]:
#tf.data API
data = pathlib.Path(r'C:\\Users\\Oguzhan\\PythonProjects\\Kiosk Proje\\speech')


In [ ]:
#checking basic statistics about the dataset
words = np.array(tf.io.gfile.listdir(str(data)))
words = words[words != '.DS_Store']
print('Words:', words)

In [ ]:
#ses dosyalarını bir listeye çıkarıp birleştirmek
filenames = tf.io.gfile.glob(str(data) + '//')
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)
print(num_samples)
#toplam 26042 örnek ve  label başına xxxx örnek

In [ ]:
#80:10:10 bölmek (eğitim, doğrulama,test)
train_files = filenames[:15626]
val_files = filenames[15625: 15624 + 5208]
test_files = filenames[-5208:]

print('train size:',len(train_files))
print('val size:',len(val_files))
print('test size:',len(test_files))

In [ ]:
#ses dosyasını sayısal bir tensöre dönüştürmeliyiz
#wav kodlu sesi , tensör ve örnekleme hızı olarak döndüren tf.audio.decode_wav'ı kullanabiliriz. => değerleri [-1.0,1.0] aralığında normalleştirir.
def decode_audio(audio_binary):
  audio, _ =tf.audio.decode_wav(audio_binary)
  return tf.squeeze(audio, axis = -1)
  


# #tf.squeeze: Given a tensor input, this operation returns a tensor of the same type with all dimensions of size 1 removed. If you don't want to remove all size 1 dimensions, you can remove specific size 1 dimensions by specifying axis.

In [ ]:
#her wav dosyasının etiketi, üst dizinidir
def get_label(file_path):
  parts = tf.strings.split(file_path, os.path.sep)
  #Not: Bunun bir TensorFlow grafiğinde çalışmasını sağlamak için tuple paketini açmak yerine burada indekslemeyi kullan
  return parts[-2]

In [ ]:
#denetimli eğitim için ses ve etiketleri içeren bir demet çıktısını alacak yöntem
def get_wave_and_label(file_path):
  label = get_label(file_path)
  audio_binary = tf.io.read_file(file_path)
  waveform = decode_audio(audio_binary)
  return waveform,label

In [ ]:
#ses etiketi çiftlerini çıkarmak, sonuçları kontrol etmek, eğitim setini oluşturmak için
AUTOTUNE = tf.data.experimental.AUTOTUNE
files_ds = tf.data.Dataset.from_tensor_slices(train_files)
waveform_ds = files_ds.map(get_wave_and_label, num_parallel_calls = AUTOTUNE)

In [ ]:
rows = 3
cols = 3
n = rows*cols 
fig, axes =plt.subplots(rows,cols, figsize=(10,12))
for i,(audio,label) in enumerate(waveform_ds.take(9)):
  r = i // cols
  c = i % cols
  ax = axes[r][c]
  ax.plot(audio.numpy())
  ax.set_yticks(np.arange(-1.2, 1.2, 0.2))
  label = label.numpy().decode('utf-8')
  ax.set_title(label)
plt.show()

# #Dalga biçimini, zaman içindeki frekans değişikliklerini gösteren ve 2B görüntü olarak temsil edilebilen bir spektrograma dönüştüreceksiniz. Bu, sesi zaman frekans alanına dönüştürmek için kısa süreli Fourier dönüşümü (STFT) uygulanarak yapılabilir.Bir Fourier dönüşümü ( tf.signal.fft ) bir sinyali bileşen frekanslarına dönüştürür, ancak tüm zaman bilgilerini kaybeder. STFT ( tf.signal.stft ) sinyali zaman pencerelerine böler ve her pencerede bir Fourier dönüşümü çalıştırır, bazı zaman bilgilerini korur ve üzerinde standart konvolüsyonlar çalıştırabileceğiniz bir 2D tensör döndürür.STFT, büyüklüğü ve fazı temsil eden bir dizi karmaşık sayı üretir. Ancak, bu öğretici için yalnızca, tf.abs çıktısına tf.abs uygulanarak elde edilebilecek büyüklüğe ihtiyacınız tf.signal.stft .
# 
# frame_length ve frame_step parametrelerini, oluşturulan spektrogram "görüntü" hemen hemen kare olacak şekilde seçin.Ayrıca dalga formlarının aynı uzunlukta olmasını istersiniz, böylece onu bir spektrogram görüntüsüne dönüştürdüğünüzde, sonuçlar benzer boyutlara sahip olacaktır. Bu, bir saniyeden daha kısa olan ses kliplerini basitçe sıfır doldurarak yapılabilir.

In [ ]:
def get_spectogram(waveform):
  #16000' den az sayıda örnek içeren dosyalar için padding
  zero_padding = tf.zeros([16000]-tf.shape(waveform), dtype = tf.float32)
  #sesleri padding ile birleştir böylece tüm ses klipleri aynı uzunlukta olacaktır
  waveform = tf.cast(waveform, tf.float32)
  equal_length = tf.concat([waveform, zero_padding], 0)
  #abs metoduna verebilmek için stft metodundan geşirdik
  spectogram = tf.signal.stft(equal_length,frame_length=255, frame_step = 128)
  spectogram = tf.abs(spectogram)
  return spectogram


In [ ]:
#veri kümesinden bir örneğin dalga biçimini, spektogramını ve gerçek sesini karşılaştıralım
for waveform,label in waveform_ds.take(1):
  label = label.numpy().decode('utf-8')
  spectogram = get_spectogram(waveform)

  print('label:',label)
  print('waveform shape:', waveform.shape)
  print('spectogram shape:',spectogram.shape)
  print('play audio')
  display.display(display.Audio(waveform, rate = 16000))

In [ ]:
#waveform ve spectogramı grafiksel olarak göstermek
def plot_spectogram(spectogram, ax):
  #zamanın x ekseninde temsil edilmesi için frekansları log ölçeğine dönüştürün ve aktarın
  log_spec = np.log(spectogram.T)
  height = log_spec.shape[0]
  #belirli bir aralıkta eşit aralıklı değerleri döndür
  X = np.arange(16000, step = height+1)
  Y = range(height)
  #Create a pseudocolor plot with a non-regular rectangular grid.
  ax.pcolormesh(X,Y,log_spec)

In [ ]:
fig,axes = plt.subplots(2, figsize=(12,8))
timescale = np.arange(waveform.shape[0])
axes[0].plot(timescale, waveform.numpy())
axes[0].set_title('Waveform')
axes[0].set_xlim([0,16000])
plot_spectogram(spectogram.numpy(), axes[1])
axes[1].set_title('Spectogram')
plt.show()

In [ ]:
#dalga formu veri setinin spektogram görüntülerine karşılık gelen etiketlerine tamsayı kimlikler atalım
def get_spectogram_and_label_id(audio,label):
  spectogram = get_spectogram(audio)
  spectogram = tf.expand_dims(spectogram, -1)
  #Returns the index with the largest value across axes of a tensor
  label_id = tf.argmax(label == words)
  return spectogram, label_id

spectogram_ds = waveform_ds.map(get_spectogram_and_label_id, num_parallel_calls=AUTOTUNE)

In [ ]:
#veri kümesinin farklı örnekleri için spektogram görüntülerini inceleme
rows = 3
cols = 3
n = rows * cols
fig,axes = plt.subplots(rows,cols,figsize = (10,10))
for i, (spectogram, label_id) in enumerate(spectogram_ds.take(n)):
  r = i//cols
  c = i % cols
  ax = axes[r][c]
  plot_spectogram(np.squeeze(spectogram.numpy()),ax)
  ax.set_title(words[label_id.numpy()])
  ax.axis('off')

plt.show()

In [ ]:
#modeli eğitmeden önce doğrulama ve test setlerinde eğitim seti ön işlemesini tekrarlamalıyız
def preprocess_dataset(files):
  files_ds = tf.data.Dataset.from_tensor_slices(files)
  output_ds = files_ds.map(get_wave_and_label, num_parallel_calls=AUTOTUNE)
  output_ds = output_ds.map(get_spectogram_and_label_id, num_parallel_calls = AUTOTUNE)
  return output_ds

In [ ]:
train_ds = spectogram_ds
val_ds = preprocess_dataset(val_files)
test_ds = preprocess_dataset(test_files)

In [ ]:
#modeli eğitmek için eğitim ve doğrulama setlerini gruplama
train_ds = train_ds.batch(batch_size=64)
val_ds = val_ds.batch(batch_size=64)

In [ ]:
#modeli eğitirken okuma gecikmesini azaltmak için cache() ve prefetch() işlemi uygula
train_ds = train_ds.cache().prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)

In [ ]:
for spectogram, _ in spectogram_ds.take(1):
  input_shape = spectogram.shape
print('Input shape:', input_shape)
num_labels = len(words) 

norm_layer = preprocessing.Normalization()
norm_layer.adapt(spectogram_ds.map(lambda x, _:x))

model = models.Sequential([
        layers.Input(shape=input_shape),
        preprocessing.Resizing(32,32),
        norm_layer,
        layers.Conv2D(32,3, activation = 'relu'),
        #layers.MaxPooling2D(),
        layers.Conv2D(64,3,activation = 'relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation = 'relu'),
        layers.Dropout(0.5),
        layers.Dense(num_labels)
])
model.summary()


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics = ['accuracy'])

In [ ]:
history = model.fit(train_ds,validation_data=val_ds,epochs=8,callbacks=tf.keras.callbacks.EarlyStopping(verbose=0,patience=2))

In [ ]:
model.save("Capstone.h5")
from tensorflow.keras.models import load_model
new_model = load_model('Capstone.h5')
new_model.summary()
new_model.get_weights()
new_model.optimizer
#Convert to .h5 file to .tflite file
model = tf.keras.models.load_model('C:\\Users\\Alperen\\PythonProjects\\Kiosk Proje\\Capstone.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("Capstone.tflite", "wb").write(tflite_model)

In [ ]:
#eğitim ve doğrulama kaybı eğrileri
metrics = history.history
plt.plot(history.epoch, metrics['loss'],metrics['val_loss'])
plt.legend(['loss','val_loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
#modeli test seti üzerinde çalıştırıp,performansı kontrol etme
test_audio = []
test_labels=[]
for audio,label in test_ds:
  test_audio.append(audio.numpy())
  test_labels.append(label.numpy())

test_audio = np.array(test_audio)
test_labels = np.array(test_labels)

y_pred = np.argmax(model.predict(test_audio),axis=1)
y_true = test_labels
test_acc = sum(y_pred == y_true) / len(y_true)
print(f'Test accuracy: {test_acc:.0%}')

In [ ]:
#karışıklık matrisi => modelin test setindeki komutların her birindeki başarısını gösterir
confusion_mtx = tf.math.confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10,8))
sb.heatmap(confusion_mtx, xticklabels=words, yticklabels=words, annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()

In [ ]:
fs=16000
duration = 1  # seconds
filename = 'record.wav'
print("Recording Audio...")
myrecording = sd.rec(duration * fs , samplerate=fs, channels=1,dtype='float64')
sd.wait()
sf.write(filename, myrecording, fs)
print("Audio recording complete")

samples, fs = librosa.load('record.wav',sr=44100)
ipd.Audio(samples,rate=44100)

data_way = pathlib.Path('C:\\Users\\Alperen\\PythonProjects\\Kiosk Proje')
sample_file = data_way/'record.wav' #gürültülü seslerle sorunsuz çalışıyor
sample_ds = preprocess_dataset([str(sample_file)])
for spectogram,label in sample_ds.batch(1):
    prediction = model(spectogram)
    plt.figure(figsize=(11,7))
    plt.bar(words,tf.nn.softmax(prediction[0]))
    plt.title(f'Prediction for "{words[label[0]]}"')
    plt.show()



print(label[0])
print('*******')
print(words,prediction[0])
maxElement = np.amax(prediction)
print(str(maxElement))
##location = prediction.tf.where(maxElement)
##print(location)

f = open("words.txt","w+")
f.write(str(filename))
f.close()